In [ ]:
!pip install datasets spacy nltk --quiet
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda

In [ ]:
import pandas as pd
import re
import spacy
import nltk
from nltk.corpus import stopwords
from string import punctuation
from datasets import load_dataset
from collections import Counter
import json


nltk.download("stopwords")
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'https?://\S+', '', text)         # removing URLs
    text = re.sub(r'[^a-z\s]', '', text)             # keeping only a-z
    doc = nlp(text)
    tokens = [
        token.lemma_ for token in docs
        if token.is_alpha and token.text not in stop_words and token.text not in punctuation
    ]
    return tokens


In [ ]:

df = pd.read_csv("data.csv")
df["tokens"] = df["Sentence"].apply(preprocess_text)

print(f"✅ Your CSV: {len(df)} rows")
df[["Sentence", "tokens"]].head()


json_data = df.to_json(orient="records", indent=2)
with open("preprocessed_data.json", "w") as f:
    f.write(json_data)

print("✅ JSON file saved as 'preprocessed_data.json'")


✅ Your CSV: 5842 rows
✅ JSON file saved as 'preprocessed_data.json'


In [ ]:
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd

ds = load_dataset("StephanAkkerman/stock-market-tweets-data")
hf_data = ds["train"]

print(f"Loaded {len(hf_data)} rows from Hugging Face dataset")

hf_tokens = []
for item in tqdm(hf_data, desc="Preprocessing FULL HuggingFace dataset"):
    tokens = preprocess_text(item["text"])
    if tokens:
        hf_tokens.append(tokens)

hf_df = pd.DataFrame([{"tokens": tokens} for tokens in hf_tokens])
print(f"✅ Hugging Face dataset (preprocessed): {len(hf_df)} rows")
hf_df.head()




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

stock-market-tweets-data.csv:   0%|          | 0.00/175M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/923673 [00:00<?, ? examples/s]

Loaded 923673 rows from Hugging Face dataset


Preprocessing FULL HuggingFace dataset: 100%|██████████| 923673/923673 [2:32:54<00:00, 100.68it/s]


✅ Hugging Face dataset (preprocessed): 923664 rows


,tokens
0,"[kennydegu, little, volume, think, could, spx,..."
1,"[esf, achieve, target, closing, fibonacci, lev..."
2,"[rt, kimblecharte, silvergold, indicator, crea..."
3,"[issaquahfund, hedge, msft, position, close, s..."
4,"[rt, zipillinois, surprisingly, controversial,..."


In [ ]:
combined_df = pd.concat([df[["tokens"]], hf_df], ignore_index=True)
print(f"🔗 Combined dataset: {len(combined_df)} sentences")


🔗 Combined dataset: 929506 sentences


In [ ]:
all_tokens = [token for tokens in combined_df["tokens"] for token in tokens]
word_freq = Counter(all_tokens)

MIN_FREQ = 5 # standard value
filtered_tokens = {word: freq for word, freq in word_freq.items() if freq >= MIN_FREQ}

vocab = sorted(filtered_tokens.keys())
word2idx = {word: idx for idx, word in enumerate(vocab)}
idx2word = {idx: word for word, idx in word2idx.items()}
vocab_size = len(vocab)

print(f"🧠 Vocab size: {vocab_size}")

with open("vocab.json", "w") as f:
    json.dump({"word2idx": word2idx, "idx2word": idx2word}, f, indent=2)

print("📁 Saved vocab.json")


🧠 Vocab size: 52280
📁 Saved vocab.json


In [ ]:
import json

with open("vocab.json", "r") as f:
    vocab_data = json.load(f)

# Displaying the data
print(vocab_data)

{'word2idx': {'aa': 0, 'aaa': 1, 'aaaa': 2, 'aaaaand': 3, 'aaaamhim': 4, 'aaaand': 5, 'aaamportfolio': 6, 'aaand': 7, 'aaarate': 8, 'aaatech': 9, 'aabb': 10, 'aacg': 11, 'aacr': 12, 'aadr': 13, 'aaf': 14, 'aagave': 15, 'aagha': 16, 'aaii': 17, 'aaiisentiment': 18, 'aaj': 19, 'aajtak': 20, 'aakhrisauda': 21, 'aakritibhalla': 22, 'aal': 23, 'aali': 24, 'aalpha': 25, 'aame': 26, 'aampb': 27, 'aan': 28, 'aandelen': 29, 'aane': 30, 'aaoi': 31, 'aaol': 32, 'aaon': 33, 'aap': 34, 'aapkiapnigullak': 35, 'aapl': 36, 'aaplc': 37, 'aaple': 38, 'aaplnvda': 39, 'aaplo': 40, 'aaplp': 41, 'aapls': 42, 'aaplta': 43, 'aapltree': 44, 'aar': 45, 'aaron': 46, 'aaronback': 47, 'aaronbasile': 48, 'aaronbush': 49, 'aarongreenspan': 50, 'aaronkleinx': 51, 'aaronmiles': 52, 'aarons': 53, 'aaronvalue': 54, 'aarp': 55, 'aarti': 56, 'aartidrug': 57, 'aartiindustrie': 58, 'aasthaaneja': 59, 'aat': 60, 'aau': 61, 'aav': 62, 'aave': 63, 'aaveaave': 64, 'aawc': 65, 'aaww': 66, 'aax': 67, 'aaxj': 68, 'aaxn': 69, 'aaz'

In [ ]:
from google.colab import files

In [ ]:
files.download('vocab.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>